In [1]:
from __future__ import print_function, division

from keras.datasets import fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 7 * 7, input_dim=self.latent_dim))
        model.add(Reshape((7,7, 512)))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3,strides=1,padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,strides=1, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, strides=2,kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(self.img_rows*self.img_cols*self.channels, input_dim=np.prod(self.img_shape)))
        model.add(Reshape((self.img_rows,self.img_cols,self.channels)))

        model.add(Conv2D(64, kernel_size=3, strides=2,padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = fashion_mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_cgan_fashion_mnist'):
            os.mkdir('images_cgan_fashion_mnist')
        fig.savefig("images_cgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=40, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.234374, acc.: 17.97%] [G loss: 0.947227]
epoch:0 step:2 [D loss: 0.336171, acc.: 80.47%] [G loss: 2.773298]
epoch:0 step:3 [D loss: 0.102709, acc.: 98.44%] [G loss: 3.755343]
epoch:0 step:4 [D loss: 0.194898, acc.: 91.41%] [G loss: 3.572041]
epoch:0 step:5 [D loss: 0.213105, acc.: 92.97%] [G loss: 3.598420]
epoch:0 step:6 [D loss: 0.050351, acc.: 100.00%] [G loss: 3.506066]
epoch:0 step:7 [D loss: 0.082331, acc.: 99.22%] [G loss: 3.414533]
epoch:0 step:8 [D loss: 0.278439, acc.: 88.28%] [G loss: 4.919701]
epoch:0 step:9 [D loss: 0.315177, acc.: 86.72%] [G loss: 6.049656]
epoch:0 step:10 [D loss: 0.255245, acc.: 90.62%] [G loss: 5.389141]
epoch:0 step:11 [D loss: 0.176346, acc.: 92.97%] [G loss: 6.052828]
epoch:0 step:12 [D loss: 0.454458, acc.: 81.25%] [G loss: 6.526584]
epoch:0 step:13 [D loss: 0.675105, acc.: 72.66%] [G loss: 5.619236]
epoch:0 step:14 [D loss: 0.654224, acc.: 71.09%] [G loss: 5.514531]
epoch:0 step:15 [D loss: 0.489142, acc.: 80.47%] [G loss

epoch:0 step:121 [D loss: 0.062799, acc.: 99.22%] [G loss: 8.141939]
epoch:0 step:122 [D loss: 0.012368, acc.: 100.00%] [G loss: 6.363040]
epoch:0 step:123 [D loss: 0.052279, acc.: 98.44%] [G loss: 5.787856]
epoch:0 step:124 [D loss: 0.051564, acc.: 99.22%] [G loss: 6.192499]
epoch:0 step:125 [D loss: 0.316352, acc.: 93.75%] [G loss: 8.801474]
epoch:0 step:126 [D loss: 1.003181, acc.: 79.69%] [G loss: 4.562503]
epoch:0 step:127 [D loss: 1.569089, acc.: 83.59%] [G loss: 8.332919]
epoch:0 step:128 [D loss: 0.379794, acc.: 89.84%] [G loss: 7.376862]
epoch:0 step:129 [D loss: 0.534898, acc.: 87.50%] [G loss: 7.084747]
epoch:0 step:130 [D loss: 0.589072, acc.: 92.19%] [G loss: 9.820095]
epoch:0 step:131 [D loss: 0.334026, acc.: 94.53%] [G loss: 6.171810]
epoch:0 step:132 [D loss: 0.545914, acc.: 86.72%] [G loss: 7.869609]
epoch:0 step:133 [D loss: 0.063916, acc.: 97.66%] [G loss: 9.714008]
epoch:0 step:134 [D loss: 0.223990, acc.: 96.09%] [G loss: 10.179804]
epoch:0 step:135 [D loss: 0.1868

...

epoch:39 step:36917 [D loss: 0.244721, acc.: 90.62%] [G loss: 7.080432]
epoch:39 step:36918 [D loss: 0.082192, acc.: 97.66%] [G loss: 2.914618]
epoch:39 step:36919 [D loss: 0.320487, acc.: 82.81%] [G loss: 4.477646]
epoch:39 step:36920 [D loss: 0.064131, acc.: 100.00%] [G loss: 6.139784]
epoch:39 step:36921 [D loss: 0.007716, acc.: 100.00%] [G loss: 5.686540]
epoch:39 step:36922 [D loss: 0.024302, acc.: 100.00%] [G loss: 1.876240]
epoch:39 step:36923 [D loss: 1.784943, acc.: 50.00%] [G loss: 3.795371]
epoch:39 step:36924 [D loss: 0.458247, acc.: 74.22%] [G loss: 1.264964]
epoch:39 step:36925 [D loss: 0.128614, acc.: 99.22%] [G loss: 1.645014]
epoch:39 step:36926 [D loss: 0.309360, acc.: 86.72%] [G loss: 3.305944]
epoch:39 step:36927 [D loss: 0.113577, acc.: 98.44%] [G loss: 0.592482]
epoch:39 step:36928 [D loss: 0.181247, acc.: 96.88%] [G loss: 0.398572]
epoch:39 step:36929 [D loss: 1.753644, acc.: 47.66%] [G loss: 4.949710]
epoch:39 step:36930 [D loss: 0.182967, acc.: 94.53%] [G loss:

epoch:39 step:37033 [D loss: 0.366424, acc.: 78.91%] [G loss: 4.654339]
epoch:39 step:37034 [D loss: 1.232648, acc.: 50.78%] [G loss: 3.174733]
epoch:39 step:37035 [D loss: 0.049935, acc.: 100.00%] [G loss: 0.983762]
epoch:39 step:37036 [D loss: 0.233154, acc.: 88.28%] [G loss: 2.991101]
epoch:39 step:37037 [D loss: 0.114723, acc.: 97.66%] [G loss: 1.359714]
epoch:39 step:37038 [D loss: 0.045545, acc.: 100.00%] [G loss: 0.218526]
epoch:39 step:37039 [D loss: 0.083472, acc.: 98.44%] [G loss: 3.662670]
epoch:39 step:37040 [D loss: 0.694690, acc.: 62.50%] [G loss: 0.343251]
epoch:39 step:37041 [D loss: 0.346057, acc.: 82.81%] [G loss: 0.133153]
epoch:39 step:37042 [D loss: 0.039874, acc.: 99.22%] [G loss: 1.698944]
epoch:39 step:37043 [D loss: 0.066032, acc.: 100.00%] [G loss: 3.337168]
epoch:39 step:37044 [D loss: 0.651064, acc.: 63.28%] [G loss: 4.737094]
epoch:39 step:37045 [D loss: 0.077283, acc.: 98.44%] [G loss: 1.526301]
epoch:39 step:37046 [D loss: 0.120718, acc.: 98.44%] [G loss:

epoch:39 step:37149 [D loss: 0.102006, acc.: 99.22%] [G loss: 3.578142]
epoch:39 step:37150 [D loss: 0.068616, acc.: 99.22%] [G loss: 6.647661]
epoch:39 step:37151 [D loss: 0.608780, acc.: 62.50%] [G loss: 4.245649]
epoch:39 step:37152 [D loss: 0.472535, acc.: 71.88%] [G loss: 0.833166]
epoch:39 step:37153 [D loss: 0.271809, acc.: 91.41%] [G loss: 4.575424]
epoch:39 step:37154 [D loss: 0.217516, acc.: 90.62%] [G loss: 3.219940]
epoch:39 step:37155 [D loss: 0.140514, acc.: 97.66%] [G loss: 3.353244]
epoch:39 step:37156 [D loss: 0.090309, acc.: 99.22%] [G loss: 7.114620]
epoch:39 step:37157 [D loss: 0.573500, acc.: 71.88%] [G loss: 3.463032]
epoch:39 step:37158 [D loss: 0.257732, acc.: 87.50%] [G loss: 7.157255]
epoch:39 step:37159 [D loss: 0.345564, acc.: 85.94%] [G loss: 5.657464]
epoch:39 step:37160 [D loss: 0.342195, acc.: 89.06%] [G loss: 3.443536]
epoch:39 step:37161 [D loss: 0.088814, acc.: 100.00%] [G loss: 5.509246]
epoch:39 step:37162 [D loss: 0.090953, acc.: 97.66%] [G loss: 6

epoch:39 step:37265 [D loss: 0.027493, acc.: 99.22%] [G loss: 4.659481]
epoch:39 step:37266 [D loss: 0.063672, acc.: 100.00%] [G loss: 5.861021]
epoch:39 step:37267 [D loss: 0.158432, acc.: 96.88%] [G loss: 4.073735]
epoch:39 step:37268 [D loss: 0.099376, acc.: 99.22%] [G loss: 7.198611]
epoch:39 step:37269 [D loss: 0.030661, acc.: 100.00%] [G loss: 0.820024]
epoch:39 step:37270 [D loss: 0.829763, acc.: 60.16%] [G loss: 4.245721]
epoch:39 step:37271 [D loss: 0.077623, acc.: 99.22%] [G loss: 9.657006]
epoch:39 step:37272 [D loss: 0.161819, acc.: 95.31%] [G loss: 4.736683]
epoch:39 step:37273 [D loss: 0.338648, acc.: 83.59%] [G loss: 0.931454]
epoch:39 step:37274 [D loss: 0.108777, acc.: 96.88%] [G loss: 5.132522]
epoch:39 step:37275 [D loss: 0.258622, acc.: 88.28%] [G loss: 6.025646]
epoch:39 step:37276 [D loss: 0.097250, acc.: 96.88%] [G loss: 1.429300]
epoch:39 step:37277 [D loss: 0.224419, acc.: 92.19%] [G loss: 5.661040]
epoch:39 step:37278 [D loss: 0.476627, acc.: 80.47%] [G loss: 

epoch:39 step:37381 [D loss: 0.091108, acc.: 100.00%] [G loss: 3.427351]
epoch:39 step:37382 [D loss: 0.042836, acc.: 100.00%] [G loss: 4.268675]
epoch:39 step:37383 [D loss: 0.115276, acc.: 98.44%] [G loss: 3.889555]
epoch:39 step:37384 [D loss: 0.301216, acc.: 92.97%] [G loss: 3.915226]
epoch:39 step:37385 [D loss: 0.090448, acc.: 98.44%] [G loss: 5.988023]
epoch:39 step:37386 [D loss: 0.037318, acc.: 99.22%] [G loss: 4.528171]
epoch:39 step:37387 [D loss: 0.012846, acc.: 100.00%] [G loss: 1.304944]
epoch:39 step:37388 [D loss: 0.122961, acc.: 97.66%] [G loss: 2.273865]
epoch:39 step:37389 [D loss: 0.049200, acc.: 100.00%] [G loss: 4.077186]
epoch:39 step:37390 [D loss: 0.080569, acc.: 99.22%] [G loss: 1.620469]
epoch:39 step:37391 [D loss: 1.030072, acc.: 41.41%] [G loss: 1.842632]
epoch:39 step:37392 [D loss: 0.301110, acc.: 87.50%] [G loss: 4.568303]
epoch:39 step:37393 [D loss: 0.033926, acc.: 100.00%] [G loss: 7.260922]
epoch:39 step:37394 [D loss: 0.125074, acc.: 95.31%] [G los